In [ ]:
tracts = gp.read_file("tracts_cle.shp")
tracts["price_to_rent"] = round(tracts.MEDIAN_HOM/(tracts.MEDIAN_CON*12),1)
tracts = tracts[tracts.geometry.area<0.2]
tracts['area'] = tracts.geometry.area

# tracts["geometry"] = tracts["geometry"].apply(lambda x: x.simplify(0.05))
tracts["geometry"] = [MultiPolygon([feature]) if type(feature) == Polygon \
    else feature for feature in tracts["geometry"]]
tracts.to_file('tracts.geojson', driver='GeoJSON')

tracts = tracts.to_crs(epsg=3857)

# bounds=geo.bounds
plot_df = tracts[tracts.geometry.area<5*10**7]
column = "price_to_rent"
vmin = plot_df[column].min()
vmax = plot_df[column].max()

ax = plot_df.plot(
    column=column,
    cmap=cmap,
#     legend=True,
    figsize=(25, 25),
    alpha=0.5,
    edgecolor='black',
    linewidth=1,
    markersize = abs(plot_df["TOTAL_POPU"]/1000*2),
#     color='red',
    scheme = 'quantiles',
)

# states.plot(
#     ax = ax,
#     linewidth=0,
#     alpha=0
# )

add_basemap(ax, zoom=1, url=ctx.sources.ST_TONER_LITE)

ax.set_axis_off()
fig = ax.get_figure()

# ax.set_xlim(-19000000, -7000000)
# ax.set_ylim(1750000, 10000000)

cbax = fig.add_axes([0.95, 0.3, 0.03, 0.39])   
cbax.set_title("Price to Rent Ratio", fontsize= 15)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
fig.colorbar(sm, cax=cbax)

matplotlib.rcParams.update({'font.size': 20})
ax.set_title('Price to Rent Ratio by City (Contiguous US)', fontdict={
    'fontsize': 30, 'fontweight': 'medium'})

# plt.savefig('Cities_Rent_to_Price.jpg',  dpi=300, bbox_inches = "tight")
plt.show()